In [2]:
!git clone https://github.com/MegaMashroomSquad/VideoTagsPred.git

Cloning into 'VideoTagsPred'...
remote: Enumerating objects: 31, done.
remote: Counting objects: 100% (31/31), done.
remote: Compressing objects: 100% (25/25), done.
remote: Total 31 (delta 11), reused 17 (delta 3), pack-reused 0 (from 0)
Receiving objects: 100% (31/31), 279.39 KiB | 3.45 MiB/s, done.
Resolving deltas: 100% (11/11), done.


In [3]:
default_path = "/kaggle/working/VideoTagsPred"

In [4]:
import pandas as pd

In [5]:
df = pd.read_csv(f"{default_path}/train_data_categories.csv")[['video_id', 'title', 'description','tags']]
taxonomy = pd.read_csv(f"{default_path}/IAB_tags.csv")


In [6]:
df['tags_split'] = df['tags'].str.split(':', expand=False)

df['tags_1'] = df['tags_split'].apply(lambda x: x[0].strip() if isinstance(x, list) and len(x) > 0 else pd.NA)
df['tags_2'] = df['tags_split'].apply(lambda x: x[1].strip() if isinstance(x, list) and len(x) > 1 else pd.NA)
df['tags_3'] = df['tags_split'].apply(lambda x: x[2].strip() if isinstance(x, list) and len(x) > 2 else pd.NA)

df = df.drop('tags_split', axis=1)

df['tags_2'] = df['tags_2'].replace('', pd.NA)
df['tags_3'] = df['tags_3'].replace('', pd.NA)

print(df[['tags', 'tags_1', 'tags_2', 'tags_3']].head())


                                                tags  \
0                   Массовая культура: Юмор и сатира   
1  События и достопримечательности:  Спортивные с...   
2  Массовая культура: Отношения знаменитостей, Ма...   
3     Транспорт, Спорт: Автогонки, Массовая культура   
4                   Массовая культура: Юмор и сатира   

                            tags_1  \
0                Массовая культура   
1  События и достопримечательности   
2                Массовая культура   
3                 Транспорт, Спорт   
4                Массовая культура   

                                       tags_2                  tags_3  
0                               Юмор и сатира                    <NA>  
1       Спортивные события, Массовая культура           Юмор и сатира  
2  Отношения знаменитостей, Массовая культура  Скандалы знаменитостей  
3                Автогонки, Массовая культура                    <NA>  
4                               Юмор и сатира                    <NA>  


In [7]:
!pip install datasets

In [8]:
import pandas as pd
import numpy as np
from datasets import Dataset, DatasetDict
from sklearn.model_selection import train_test_split


In [9]:
df = df.dropna(subset=['tags_1'])
df = df.reset_index(drop=True)

df['text'] = 'Title: ' + df['title'] + '\n\nDescription: ' + df['description']


In [10]:
single_example_classes = df['tags_1'].value_counts()[df['tags_1'].value_counts() == 1].index.tolist()

for class_name in single_example_classes:
    row_to_duplicate = df[df['tags_1'] == class_name].iloc[0]
    df = pd.concat([df, pd.DataFrame([row_to_duplicate])], ignore_index=True)

df = df.reset_index(drop=True)

print(f"Duplicated rows for {len(single_example_classes)} classes with only one example.")
print(f"New dataset size: {len(df)} rows")


Duplicated rows for 44 classes with only one example.
New dataset size: 1092 rows


In [12]:

# Create label encoding
unique_labels = sorted(df['tags_1'].unique())
label_to_id = {label: i for i, label in enumerate(unique_labels)}
labels = pd.get_dummies(df['tags_1']).astype(int)


In [13]:
# Create label weights
label_weights = 1 - labels.sum() / labels.sum().sum()


In [15]:

# Iterative train-test split for unbalanced classes
from skmultilearn.model_selection import iterative_train_test_split
import numpy as np

row_ids = np.arange(len(labels)).reshape(-1, 1)
X_train, y_train, X_val, y_val = iterative_train_test_split(row_ids, labels.values, test_size=0.1)

x_train = df['text'].iloc[X_train.flatten()].tolist()
x_val = df['text'].iloc[X_val.flatten()].tolist()

y_train = pd.DataFrame(y_train, columns=labels.columns)
y_val = pd.DataFrame(y_val, columns=labels.columns)



In [16]:

# Convert labels to list of dictionaries
y_train_dict = y_train.reset_index(drop=True).to_dict('records')
y_val_dict = y_val.reset_index(drop=True).to_dict('records')

# Create HuggingFace dataset
ds = DatasetDict({
    'train': Dataset.from_pandas(pd.DataFrame({'text': x_train, 'labels': y_train_dict})),
    'val': Dataset.from_pandas(pd.DataFrame({'text': x_val, 'labels': y_val_dict}))
})

print(f"Dataset created with {len(ds['train'])} training samples and {len(ds['val'])} validation samples.")
print(f"Number of unique labels: {len(unique_labels)}")
print(f"Label weights: {label_weights.values}")


Dataset created with 993 training samples and 99 validation samples.
Number of unique labels: 89
Label weights: [0.9981685  0.99450549 0.97985348 0.94139194 0.9981685  0.99725275
 0.99267399 0.9981685  0.9981685  0.9981685  0.9981685  0.9981685
 0.9981685  0.9981685  0.99542125 0.9981685  0.9981685  0.9981685
 0.9981685  0.99175824 0.9981685  0.9981685  0.70695971 0.99358974
 0.99450549 0.9981685  0.9981685  0.96611722 0.99450549 0.9981685
 0.99725275 0.99175824 0.99725275 0.9981685  0.9981685  0.98168498
 0.99542125 0.9981685  0.99358974 0.9981685  0.9981685  0.9981685
 0.99542125 0.9981685  0.97435897 0.9981685  0.99725275 0.9981685
 0.98168498 0.9981685  0.9981685  0.9981685  0.96336996 0.98717949
 0.9981685  0.9981685  0.9981685  0.99725275 0.98809524 0.9981685
 0.98168498 0.96611722 0.9981685  0.9981685  0.92948718 0.98168498
 0.9981685  0.9981685  0.97435897 0.9981685  0.9981685  0.9981685
 0.9532967  0.99542125 0.99175824 0.9981685  0.9981685  0.9981685
 0.99267399 0.9981685  0.

In [17]:
!pip install peft


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.5/322.5 kB 2.7 MB/s eta 0:00:0000:0100:01m


In [18]:
!pip install -q -U bitsandbytes

In [19]:
import os
import random
import functools
import csv
import numpy as np
import torch
import torch.nn.functional as F
from sklearn.metrics import f1_score
from skmultilearn.model_selection import iterative_train_test_split
from datasets import Dataset, DatasetDict
from peft import (
    LoraConfig,
    prepare_model_for_kbit_training,
    get_peft_model
)
from transformers import (
    AutoModelForSequenceClassification,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    Trainer
)

In [20]:
from huggingface_hub import login

login()

In [21]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, GemmaTokenizer

model_id = "Vikhrmodels/Vikhr-Nemo-12B-Instruct-R-21-09-24"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map={"":0})

tokenizer_config.json:   0%|          | 0.00/178k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.26M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/641 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/687 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/29.9k [00:00<?, ?B/s]

...

model-00003-of-00005.safetensors:   0%|          | 0.00/4.91G [00:00<?, ?B/s]

model-00004-of-00005.safetensors:   0%|          | 0.00/4.91G [00:00<?, ?B/s]

model-00005-of-00005.safetensors:   0%|          | 0.00/4.91G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

In [22]:
lora_config = LoraConfig(
    r=8,
    target_modules= [
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
        "lm_head",
    ],
    lora_alpha=16,
    lora_dropout=0.05,
    bias="none",
    task_type="SEQ_CLS"
)

In [25]:
model = AutoModelForSequenceClassification.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    num_labels=labels.shape[1]
)
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, lora_config)
model.config.pad_token_id = tokenizer.pad_token_id

`low_cpu_mem_usage` was None, now set to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 140.00 MiB. GPU 0 has a total capacity of 14.74 GiB of which 32.12 MiB is free. Process 5254 has 14.71 GiB memory in use. Of the allocated memory 14.48 GiB is allocated by PyTorch, and 115.78 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
def tokenize_examples(examples, tokenizer):
    tokenized_inputs = tokenizer(examples['text'])
    tokenized_inputs['labels'] = examples['labels']
    return tokenized_inputs

tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
tokenized_ds = ds.map(functools.partial(tokenize_examples, tokenizer=tokenizer), batched=True)
tokenized_ds = tokenized_ds.with_format('torch')

In [ ]:
# define custom batch preprocessor
def collate_fn(batch, tokenizer):
    dict_keys = ['input_ids', 'attention_mask', 'labels']
    d = {k: [dic[k] for dic in batch] for k in dict_keys}
    d['input_ids'] = torch.nn.utils.rnn.pad_sequence(
        d['input_ids'], batch_first=True, padding_value=tokenizer.pad_token_id
    )
    d['attention_mask'] = torch.nn.utils.rnn.pad_sequence(
        d['attention_mask'], batch_first=True, padding_value=0
    )
    d['labels'] = torch.stack(d['labels'])
    return d

# define which metrics to compute for evaluation
def compute_metrics(p):
    predictions, labels = p
    f1_micro = f1_score(labels, predictions > 0, average = 'micro')
    f1_macro = f1_score(labels, predictions > 0, average = 'macro')
    f1_weighted = f1_score(labels, predictions > 0, average = 'weighted')
    return {
        'f1_micro': f1_micro,
        'f1_macro': f1_macro,
        'f1_weighted': f1_weighted
    }


In [ ]:
from trl import SFTTrainer

# create custom trainer class to be able to pass label weights and calculate mutilabel loss
class CustomTrainer(SFTTrainer):

    def __init__(self, label_weights, **kwargs):
        super().__init__(**kwargs)
        self.label_weights = label_weights
    
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.pop("labels")
        
        # forward pass
        outputs = model(**inputs)
        logits = outputs.get("logits")
        
        # compute custom loss
        loss = F.binary_cross_entropy_with_logits(logits, labels.to(torch.float32), pos_weight=self.label_weights)
        return (loss, outputs) if return_outputs else loss


In [ ]:
# define training args
training_args = TrainingArguments(
    output_dir = 'multilabel_classification',
    learning_rate = 1e-4,
    per_device_train_batch_size = 8,
    per_device_eval_batch_size = 8,
    num_train_epochs = 10,
    weight_decay = 0.01,
    evaluation_strategy = 'epoch',
    save_strategy = 'epoch',
    load_best_model_at_end = True
)

# train
trainer = CustomTrainer(
    model = model,
    args = training_args,
    train_dataset = tokenized_ds['train'],
    eval_dataset = tokenized_ds['val'],
    tokenizer = tokenizer,
    data_collator = functools.partial(collate_fn, tokenizer=tokenizer),
    compute_metrics = compute_metrics,
    label_weights = torch.tensor(label_weights, device=model.device)
)

trainer.train()

# save model
peft_model_id = 'multilabel_mistral'
trainer.model.save_pretrained(peft_model_id)
tokenizer.save_pretrained(peft_model_id)
